In [44]:
import os
import cv2
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [111]:
gFrames = list()

def load_cache():
    vVideoCapture = cv2.VideoCapture("../resources/video/videoplayback.mp4")
    vSucces, vFrame = vVideoCapture.read()
    vIndex = 0
    try:
        while vSucces:
            vFrame = cv2.cvtColor(vFrame, cv2.COLOR_BGR2RGB)
            
            vFilesPath = [fn for fn in os.listdir(f"../resources/cache/frame_{vIndex}/") if "object_detected_" in fn]
            vBoxes = list()
            vMasks = list()
            vPoses = list()
            if len(vFilesPath) > 0:
                vIndexObject = 0
                for _ in vFilesPath:
                    # I'm a idiot that write "json" as "josn".
                    with open(f"../resources/cache/frame_{vIndex}/object_detected_{vIndexObject}.josn", "r") as tFile:
                        vBoxes.append(json.load(tFile))
                    vMasks.append(Image.open(f"../resources/cache/frame_{vIndex}/mask_{vIndexObject}.png"))
                    vPoses.append(Image.open(f"../resources/cache/frame_{vIndex}/pose_{vIndexObject}.jpg"))
                    vIndexObject = vIndexObject + 1
            
            yield {
                "frame_index": vIndex,
                "original_frame": vFrame,
                "detected_objects": vBoxes,
                "masks": vMasks,
                "poses": vPoses
            }
            vSucces, vFrame = vVideoCapture.read()
            vIndex = vIndex + 1
    finally:
        vVideoCapture.release()

def remove_mask(iImage, iMask, iInverseRemove : bool = False):
    vImage = np.array(iImage)
    vMask  = np.array(iMask)
    
    vAlphaChannel = vMask * np.ones(vImage.shape[:2], dtype = np.uint8)
    if iInverseRemove:
        vAlphaChannel = 255 - vAlphaChannel

    return Image.fromarray(np.dstack((vImage, vAlphaChannel)))

def combine_masks(iMaskList):
    vMaskResult = iMaskList[0]
    for iMask in iMaskList:
        vMaskResult = np.maximum(vMaskResult, iMask)
    return vMaskResult

def remove_masks(iImage, iMasks, iSegment = False):
    vMaskCombined = Image.fromarray(combine_masks(iMasks))
    vImageWithoutPersons = remove_mask(iImage, vMaskCombined, True)
    vResultArray = np.array(vImageWithoutPersons)
    
    if iSegment:
        vTransparentPixels = np.all(vResultArray <= [255, 255, 255, 0], axis = -1)
        vResultArray[vTransparentPixels] = (255, 0, 0, 0)
    
    return Image.fromarray(vResultArray)

def paste_pose(iImage, iPose, iBox):
    vPose = iPose.resize((iBox["width"], iBox["heith"]))
    vPoseArray = np.array(vPose)
    
    vAlphaArray = 255 + np.zeros((vPoseArray.shape[0], vPoseArray.shape[1], 1), dtype=np.uint8)
    vPoseArray = np.concatenate((vPoseArray, vAlphaArray), axis=2)
    vBlackPixels = np.all(vPoseArray <= [20, 20, 20, 255], axis = -1)
    vPoseArray[vBlackPixels, 3] = 0
    vPose = Image.fromarray(vPoseArray)

    # Crear una máscara de la imagen Pose
    vPoseMask = vPose.split()[3].resize(vPose.size)
    
    iImage.paste(vPose, (iBox['box'][0], iBox['box'][1]), vPoseMask)

def paste_poses(iImage, iPoses, iBoxes):
    for tPose, tBox in zip(iPoses, iBoxes):
        paste_pose(iImage, tPose, tBox)
    return iImage

def get_framerate_and_fourcc(iFilePath):
    vVideoCapture = cv2.VideoCapture(iFilePath)
    vFrameRate = vVideoCapture.get(cv2.CAP_PROP_FPS)
    vFourcc = int(vVideoCapture.get(cv2.CAP_PROP_FOURCC))
    vVideoCapture.release()

    return vFrameRate, vFourcc

gFrameRate, gFourcc = get_framerate_and_fourcc("../resources/video/videoplayback.mp4")

In [ ]:
vVideoWriter = cv2.VideoWriter(
    "../resources/video/video_pose.mp4",
    cv2.VideoWriter_fourcc(*'MP4V'),
    gFrameRate,
    (640, 360)
)
try:
    for vData in load_cache():
        if vData["frame_index"] == 59 and False:
            print("Mask shape:", vEndFrame.size)
            plt.figure()
            _, vAxisArray = plt.subplots(1, 5, figsize = (15, 15))
            for i, tImage in enumerate([
                vData["original_frame"],
                vData["masks"][0],
                vData["poses"][0],
                vNewFrame,
                vEndFrame
            ]):
                vAxisArray[i].imshow(tImage)
                vAxisArray[i].axis('off')
            plt.show()
            break
        vEndFrame = vData["original_frame"]
        if len(vData["detected_objects"]) > 0:
            vNewFrame = remove_masks(vData["original_frame"], vData["masks"], False)
            vEndFrame = paste_poses(vNewFrame.copy(), vData["poses"], vData["detected_objects"])
        
        vEndFrame = np.array(vEndFrame)
        vEndFrame = cv2.cvtColor(vEndFrame, cv2.COLOR_RGB2BGR)
        vVideoWriter.write(vEndFrame)
finally:
    vVideoWriter.release()